# oversampling

In [11]:
# Clone the entire repo.
!git clone -l -s git://github.com/NeilBotelho/YSP-Exoplanets.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 171 (delta 82), reused 107 (delta 38), pack-reused 0
Receiving objects: 100% (171/171), 3.58 MiB | 6.53 MiB/s, done.
Resolving deltas: 100% (82/82), done.
/content/cloned-repo/cloned-repo/cloned-repo
allFeatures.txt  exoplanets.csv     importance	    scikitImputer.py
badrecords	 expsOfSelected     imputation.R    technical.py
corr.py		 FixMiceImputed.py  investigate.py  unimputedDataset
dataset		 getexp.py	    missforest.R
datasetmaker.py  hmisc.R	    nonCategorical


## Imports:

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.svm import SVC
import random as r
from imblearn.over_sampling import SMOTE
r.seed(300)


## Make Preprocessed

In [2]:
exoData=pd.read_csv('../simpleImputedMiceRf')
#These are the row_ids of exoplanets that are known to be habitable
habitableRows=[151, 152, 153, 1604, 2155, 2223, 2882, 3133, 3606, 3716, 3742,3743, 3744]

#Creates a Series that uses 1 or 0 to indicate whether a corresponding record in the exoData
habitable=exoData.row_id.isin(habitableRows).replace(True,1).rename('habitable')

#Store row ids series and remove for scaling all data to values between 0 and 1
row_id=exoData.row_id
data=exoData.drop('row_id',axis=1)
scaledData=pd.DataFrame(StandardScaler().fit_transform(data),columns=data.columns)

#join the scaled data columns, row_id and habitable column,into the variable preprocessed and shuffle
preprocessed=pd.concat([row_id,scaledData,habitable],axis=1)
preprocessed=shuffle(preprocessed,random_state=100).reset_index()
preprocessed=preprocessed.drop('index',axis=1)   
print('Done')

Done


In [3]:
#List of columns to be used for training
#it will be all columns in preprocessed except for "habitable" and "row_id"
trainCols=[x for x in preprocessed.columns if x not in ['habitable','row_id']]
validate=[]
Hcopy=habitableRows.copy()

#numHidden sets the number of habitable planets to use for validation of the model
numHidden=round(len(Hcopy)/2)

#Randomly select "numHidden" number of habitable exoplanets and
#add their row_id to validation set 
print("Hiding ",numHidden," habitable(",end="")
for i in range(numHidden): 
    randNum=r.randint(0,len(Hcopy)-1)
    validate.append(Hcopy[randNum])
    print(Hcopy[randNum],end=",")
    del Hcopy[randNum]
print("\b )")

#Add row_id of non-habitable planets to the validation set till its length becomes 100
while len(validate)<100:
    temp=r.randint(0,3500)
    if temp not in habitableRows:
        validate.append(temp)

#Take all columns of the planets whose row_id is in "validate" variable(in the validation set) and
#store it in "validate" variable
validate=preprocessed[preprocessed.row_id.isin(validate)]

#Store the planets that are not in the validation set in the training set
trainingSet=preprocessed[~preprocessed.row_id.isin(validate.row_id)]

#Store the training features in X and target feature(habitable or not) in y 
X=trainingSet[trainCols]
y=trainingSet.habitable

#the SMOTE library mutates existing data to creating more data
#Here we use SMOTE to increase the number of habitable planets in the training and validation data
smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(X, y)
validateX,validateY=smote.fit_sample(validate[trainCols],validate.habitable)

Hiding  6  habitable(3716,2223,2882,151,3743,3606, )


In [4]:
#SVC TEST
#Find the optimal hyperparameters for SVC using the following scoring meassures
from sklearn.metrics import balanced_accuracy_score

# final_model=SVC(C=0.001,coef0=0,decision_function_shape='ovo',gamma='auto',kernel='linear',max_iter=-1,shrinking=True,tol=0.001,verbose=False,random_state=n)
#Parameters to check:
coef=[0,0.1,0.001,0.01]
kernel=['linear','rbf']
degree=[1,2,3]
C =[1,0.1,0.01,0.001,0.00001]
shrink=[True,False]
decision_shape=['ovo','ovr']
tol=[1,0.1,0.01,0.001,0.0001]

Bestscores={balanced_accuracy_score:['balanced_accuracy_score',-1,{}]}
n=0
#Training the data on some probable hyperparameters
#storing the best parameters in the Bestscores dictionary
for c in C:
    for dec in decision_shape:
        for d in degree:
            for k in kernel:
                for co in coef:
                    for t in tol:
                        for s in shrink:
                            n+=1
                            print(n,end=" ")
                            testSvc=SVC(C=c,coef0=co,tol=t,kernel=k,degree=d,gamma='auto',shrinking=s,decision_function_shape=dec)
                            testSvc.fit(X_sm,y_sm)
                            y_preds=testSvc.predict(validateX)
                            for score_method in Bestscores:
                                currScore=score_method(validateY,y_preds)
                                if(currScore>Bestscores[score_method][1]):
                                    Bestscores[score_method][1]=currScore
                                    Bestscores[score_method][2]=testSvc.get_params()

Bestscores



1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 

241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 

KeyboardInterrupt: 

In [20]:
N=-1
m=-1
for n in range(300):
    final_model=SVC(C=0.001,coef0=0,decision_function_shape='ovo',gamma='auto',kernel='linear',max_iter=-1,shrinking=True,tol=0.001,verbose=False,random_state=n)
    final_model.fit(X_sm,y_sm)
    preds=final_model.predict(validateX)
    b=balanced_accuracy_score(validateY,preds)
    print(b)
    if(b>m):
        N=n
        m=b
        print('\t',N)
b

# a=final_model.coef_

0.7758620689655172
	 0
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.7758620689655172
0.775862

KeyboardInterrupt: ignored

In [0]:
print(len(a[0]))
print(list(X.columns))

56
['pl_controvflag', 'pl_pnum', 'pl_orbper', 'pl_orbsmax', 'pl_radj', 'pl_ttvflag', 'pl_kepflag', 'pl_k2flag', 'ra', 'dec', 'st_dist', 'st_optmag', 'gaia_gmag', 'st_teff', 'st_mass', 'st_rad', 'pl_tranflag', 'pl_rvflag', 'pl_imgflag', 'pl_astflag', 'pl_omflag', 'pl_cbflag', 'pl_angsep', 'pl_rade', 'pl_rads', 'pl_trandur', 'pl_tranmid', 'pl_ratror', 'pl_mnum', 'pl_st_npar', 'pl_st_nref', 'st_rah', 'st_glon', 'st_glat', 'st_elon', 'st_elat', 'gaia_plx', 'gaia_dist', 'st_pmra', 'st_pmdec', 'st_pm', 'gaia_pmra', 'gaia_pmdec', 'gaia_pm', 'st_logg', 'st_metfe', 'st_j', 'st_h', 'st_k', 'st_wise1', 'st_wise2', 'st_wise3', 'st_wise4', 'st_jmh2', 'st_hmk2', 'st_jmk2']


In [0]:
dict={}
for n in range(len(a[0])):
    dict[a[0][n]]=list(X.columns)[n]
key=list(dict.keys())
key.sort(reverse=True)
print(dict)
for n in range(len(key)):
    print(dict[key[n]].strip(),str(key[n]).strip())

{0.06820574449305566: 'pl_controvflag', -0.06052800051156718: 'pl_pnum', -0.05726457989864165: 'pl_orbper', 0.018231054798979476: 'pl_orbsmax', -0.0762499435885903: 'pl_radj', -0.08145380181520667: 'pl_ttvflag', 0.028739054725018233: 'pl_kepflag', -0.07241572953387079: 'pl_k2flag', -0.0036373735179469767: 'ra', 0.04573789065963039: 'dec', -0.06546227392955092: 'st_dist', 0.0774403590018017: 'st_optmag', 0.07643876203071649: 'gaia_gmag', -0.004053545195593777: 'st_teff', 0.052752119518702685: 'st_mass', 0.021588550470381652: 'st_rad', -0.01658526407552571: 'pl_tranflag', 0.013636926771971104: 'pl_rvflag', -0.05230707219224266: 'pl_imgflag', 7.806255641895632e-18: 'pl_astflag', -0.01128086805312142: 'pl_omflag', -0.015921808193265653: 'pl_cbflag', 0.19840467037606832: 'pl_angsep', -0.08596210536295704: 'pl_rade', -0.0872467537849364: 'pl_rads', 0.14692448022683485: 'pl_trandur', 0.12365154452238862: 'pl_tranmid', -0.06718598410110924: 'pl_ratror', 0.0: 'pl_mnum', 0.14449642891955602: 'pl

In [0]:
print(len(a))

NameError: ignored

In [0]:
a=pd.Series(finalSVC.coef_[0])
b=pd.Series(X.columns)
print(type(b))
feature_importance=pd.DataFrame(dict(s1=a,s2=b)).rename(columns={'s2':"feature",'s1':"importance"})
# c=pd.concat([a[0],b],axis=1)
# len(X.columns)==len(a[0])
# feature_importance=pd.DataFrame(np.vstack((np.array(X.columns), a[0]))).transpose().rename(columns={0:"feature",1:"importance"})
# # feature_importance['importance']=pd.to_numeric(feature_importance['importance'])
# feature_importance.importance=a[0]
feature_importance.sort_values(by='importance', ascending=False)
# feature_importance.columns

<class 'pandas.core.series.Series'>


,importance,feature
41,4.428936e-01,gaia_pmra
22,3.094897e-01,pl_angsep
34,3.080106e-01,st_elon
36,2.864211e-01,gaia_plx
29,2.585763e-01,pl_st_npar
27,2.352797e-01,pl_ratror
0,1.928715e-01,pl_controvflag
11,1.907070e-01,st_optmag
6,1.696244e-01,pl_kepflag
49,1.503907e-01,st_wise1
